# Read a XML file
cool point: light file, contain the taxonomy, DNA sequence and a lot of other info

In [1]:
import xml.etree.ElementTree as ET
tree = ET.parse('sequence.gbc.xml')
root = tree.getroot()

In [2]:
tax = root.find("./INSDSeq/INSDSeq_taxonomy").text
print(">", tax)

seq = root.find("./INSDSeq/INSDSeq_sequence").text
print(seq)

> Bacteria; Bacillota; Bacilli; Bacillales; Bacillaceae; Anaerobacillus
atgcgtcaaatagctttctacggtaagggtggtattggtaagtctacgacttctcaaaatacgttagttcaattagcaaaaaattatggtcaagaaatcatgatcgttggttgtgacccgaaggctgactctacgagactagtattaaatacaaaaatgcaacaaactgttttatctttagctgctgaaaaaggaacagttgaagatttagaactacatgaagtagcgcaaaatggttttgctggcattgaatgtgttgagtctggtgggccagaaccaggtgtagggtgtgcaggtcgaggaattatcacatctattaacttccttgaagaagaaggcgcttatgaaggaaaagactttatttcttatgacgtattaggggacgttgtttgtggtggatttgcgatgccaatccgtgaaaataaagcccaagaaatttacatcgtatgttctggtgaaatgatggcaatgtacgcagctaacaacattgcaaaaggggtattaaaatatgcacacagtggtggcgttcgtttaggtggtttaatttgtaacagccgtaacacagaccgagaagcagaccttattacagagttagcaagacgattaaatactcaaatgatccattttgttcctcgtgacaatattgtacagcatgctgagttaagaagaatgacagttgctcaatatgctcctgaaagtcaacaagcaaaagagtacgatatgttagctaagaaaattttagataatgatatgatgacaattcctacgccaatcgaaactgatgagttagaaggactattaatggaattcggtattattgaggatgaagaaactcaacttaaaaaattagcagcaaaagaagcagctagtcagtaa


# Search in IPG

## Search ID

In [3]:
# Import packages
from Bio import Entrez
from Bio import SeqIO

Entrez.email = "remi.legrand38@gmail.com.com"

In [4]:
# function to search data on NCBI
def search_id(query,number_seq):
    handle = Entrez.esearch(db="ipg", term=query, retmax=number_seq) # number of sequences
    record = Entrez.read(handle)
    handle.close()
    return record["IdList"]

In [50]:
# create a list containing the ID
query = "((nifH[Gene Name]) AND 100:350[Sequence Length]) NOT uncultured"
seq_id = search_id(query,20000)
print(seq_id)
len(seq_id)

['127971653', '127920796', '127919708', '127531281', '127125494', '127122484', '124605006', '123699430', '123240796', '123238747', '122968989', '15644230', '651109310', '651099322', '539807534', '427539919', '427349194', '427347870', '427347502', '427307658', '123485017', '120090222', '108730382', '100437320', '77311835', '77141912', '75522125', '75003110', '68375739', '56757924', '56229910', '55856836', '55125244', '52011023', '46203031', '44492778', '41802330', '41802272', '41801635', '40163719', '32423034', '32005358', '32004869', '31938024', '20398129', '18484032', '15422654', '444818', '296546', '173295', '107895', '94880', '568627389', '427352784', '427352231', '427323180', '427300029', '427299397', '349975272', '328571347', '319883179', '319881941', '240261437', '238099139', '227006154', '214571543', '214571338', '200599404', '199647670', '195996521', '195508867', '195498420', '194947467', '178280531', '159366625', '152984701', '152983676', '152982965', '152982650', '152982176',

3944

At least I obtain the right ID for the IPG database

## Retrieve info for ID

In [79]:
from Bio import Entrez

# ipg_accession = "127971653"  # Replace with the desired IPG accession number

def ipg_report(ipg_accession):
    # Use the efetch function to retrieve the IPG record in XML format
    handle = Entrez.efetch(db="ipg", id=ipg_accession, rettype="ipg", retmode="text")
    ipg_record = handle.read()

    # Save the IPG record to a file
    filename = f"IPG_report/IPG_{ipg_accession}.txt"
    with open(filename, "wb") as file:
        file.write(ipg_record)

    print(f"IPG record saved to '{filename}'")

for id in seq_id:
    ipg_report(id)
    # print(id)
# ipg_report('127919708')

IPG record saved to 'IPG_report/IPG_127971653.txt'
IPG record saved to 'IPG_report/IPG_127920796.txt'
IPG record saved to 'IPG_report/IPG_127919708.txt'
IPG record saved to 'IPG_report/IPG_127531281.txt'
IPG record saved to 'IPG_report/IPG_127125494.txt'
IPG record saved to 'IPG_report/IPG_127122484.txt'
IPG record saved to 'IPG_report/IPG_124605006.txt'
IPG record saved to 'IPG_report/IPG_123699430.txt'
IPG record saved to 'IPG_report/IPG_123240796.txt'
IPG record saved to 'IPG_report/IPG_123238747.txt'
IPG record saved to 'IPG_report/IPG_122968989.txt'
IPG record saved to 'IPG_report/IPG_15644230.txt'
IPG record saved to 'IPG_report/IPG_651109310.txt'
IPG record saved to 'IPG_report/IPG_651099322.txt'
IPG record saved to 'IPG_report/IPG_539807534.txt'
IPG record saved to 'IPG_report/IPG_427539919.txt'
IPG record saved to 'IPG_report/IPG_427349194.txt'
IPG record saved to 'IPG_report/IPG_427347870.txt'
IPG record saved to 'IPG_report/IPG_427347502.txt'
IPG record saved to 'IPG_report/

In [203]:
import pandas as pd
import numpy as np
link = list()



ipg = pd.read_csv('IPG_report/IPG_851738.txt', sep="\t")

insdc = np.array(ipg.loc[:,"Source"] == "INSDC", dtype="bool")
refseq = np.array(ipg.loc[:,"Source"] == "RefSeq", dtype="bool")

in_ncbi = ipg.loc[refseq | insdc,:]
url = str()
# in_ncbi.reindex(range(0,len(in_ncbi)))
in_ncbi = in_ncbi.reset_index()

if len(in_ncbi) != 0:
    name = str(in_ncbi.loc[0,"Nucleotide Accession"])
    start = str(in_ncbi.loc[0,"Start"])
    stop = str(in_ncbi.loc[0,"Stop"])
    if str(in_ncbi.loc[0,"Strand"]) == "+":
        strand = "1"
    elif str(in_ncbi.loc[0,"Strand"]) == "-":
        strand = "2"

    url = "https://ncbi.nlm.nih.gov/nuccore/"+name+"?from="+start+"&to="+stop+"&strand="+strand
    link.append(url)
else:
    print("not in NCBI")
print(link)

['https://ncbi.nlm.nih.gov/nuccore/Z31716.1?from=527.0&to=1414.0&strand=1']


In [206]:
import pandas as pd
import numpy as np

link = list()

for name in seq_id:
    file = "IPG_report/IPG_"+name+".txt"
    ipg = pd.read_csv(file, sep="\t")

    insdc = np.array(ipg.loc[:,"Source"] == "INSDC", dtype="bool")
    refseq = np.array(ipg.loc[:,"Source"] == "RefSeq", dtype="bool")

    in_ncbi = ipg.loc[refseq | insdc,:]
    in_ncbi = in_ncbi.reset_index()
    url = str()
    if len(in_ncbi) != 0:
        # print(file)
        name = str(in_ncbi.loc[0,"Nucleotide Accession"])
        start = str(in_ncbi.loc[0,"Start"])
        stop = str(in_ncbi.loc[0,"Stop"])
        if str(in_ncbi.loc[0,"Strand"]) == "+":
            strand = "1"
        elif str(in_ncbi.loc[0,"Strand"]) == "-":
            strand = "2"

        url = "https://ncbi.nlm.nih.gov/nuccore/"+name+"?from="+start+"&to="+stop+"&strand="+strand
        link.append(url)
    else:
        print(file+" is not in NCBI")

print(link)

IPG_report/IPG_418561256.txt is not in NCBI
IPG_report/IPG_544922389.txt is not in NCBI
['https://ncbi.nlm.nih.gov/nuccore/NZ_MLQR01000001.1?from=372833&to=373708&strand=2', 'https://ncbi.nlm.nih.gov/nuccore/NZ_FOTS01000001.1?from=93697&to=94572&strand=1', 'https://ncbi.nlm.nih.gov/nuccore/NZ_FOTS01000008.1?from=37996&to=38739&strand=2', 'https://ncbi.nlm.nih.gov/nuccore/NZ_FORX01000022.1?from=18558&to=19388&strand=2', 'https://ncbi.nlm.nih.gov/nuccore/NZ_FOTI01000014.1?from=22821&to=23660&strand=2', 'https://ncbi.nlm.nih.gov/nuccore/NZ_FOVE01000009.1?from=53785&to=54663&strand=2', 'https://ncbi.nlm.nih.gov/nuccore/NZ_FNOW01000025.1?from=23764&to=24648&strand=2', 'https://ncbi.nlm.nih.gov/nuccore/NZ_FNAM01000008.1?from=13134&to=13949&strand=2', 'https://ncbi.nlm.nih.gov/nuccore/NZ_FNHB01000001.1?from=1102358&to=1103131&strand=2', 'https://ncbi.nlm.nih.gov/nuccore/NZ_FNHB01000005.1?from=87072&to=87845&strand=2', 'https://ncbi.nlm.nih.gov/nuccore/NZ_FNHQ01000044.1?from=8938&to=9699&stran

In [208]:
file = open('links.txt','w')
for item in link:
	file.write(item+"\n")
file.close()

In [60]:
seq_id[1]

'127920796'

In [219]:
import requests

url = "https://www.ncbi.nlm.nih.gov/nuccore/FOVE01000009.1?from=53785&to=54663&strand=2"
file_number = 0
for url in link:
    # Send a GET request to the URL
    response = requests.get(url)
    file_number += 1
    # Check if the request was successful (status code 200)
    if response.status_code == 200:
        # Extract the content of the response (XML file)
        xml_content = response.text
        file_name = f"xml_file/file_{file_number}.xml"
        # Save the XML content to a file
        with open(file_name, "w") as file:
            file.write(xml_content)
            
        print(file_number," downloaded successfully.")
    else:
        print("Failed to download the XML file.")


1  downloaded successfully.
2  downloaded successfully.
3  downloaded successfully.
4  downloaded successfully.
5  downloaded successfully.
6  downloaded successfully.
7  downloaded successfully.
8  downloaded successfully.
9  downloaded successfully.
10  downloaded successfully.
11  downloaded successfully.
12  downloaded successfully.
13  downloaded successfully.
14  downloaded successfully.
15  downloaded successfully.
16  downloaded successfully.
17  downloaded successfully.
18  downloaded successfully.
19  downloaded successfully.
20  downloaded successfully.
21  downloaded successfully.
22  downloaded successfully.
23  downloaded successfully.
24  downloaded successfully.
25  downloaded successfully.
26  downloaded successfully.
27  downloaded successfully.
28  downloaded successfully.
29  downloaded successfully.
30  downloaded successfully.
31  downloaded successfully.
32  downloaded successfully.
33  downloaded successfully.
34  downloaded successfully.
35  downloaded successf

In [217]:
import requests

url = "https://www.ncbi.nlm.nih.gov/nuccore/NZ_MLQR01000001.1?from=372833&to=373708&strand=2"

# Send a GET request to the URL
response = requests.get(url)

# Check if the request was successful (status code 200)
if response.status_code == 200:
    # Extract the content of the response (XML file)
    xml_content = response.text

    # Save the XML content to a file
    with open("xml_file/result.xml", "w") as file:
        file.write(xml_content)
        
    print("XML file downloaded successfully.")
else:
    print("Failed to download the XML file.")

XML file downloaded successfully.
